# Python & DuckDB

DuckDB is an embedded, high-performance analytical database designed for fast, in-memory processing of large datasets. It's similar to SQLite, but optimized for analytical queries rather than transactional ones.
Key Features of DuckDB in Python:

-   Fast Query Execution: Optimized for OLAP (Online Analytical Processing) workloads.
-    In-Memory Processing: Runs directly in memory, making it much faster than disk-based databases for analytics.
-    Seamless Pandas Integration: Easily reads/writes Pandas DataFrames.
-    SQL-Based Interface: You can use standard SQL queries in Python without setting up a database server.
-    Lightweight & Serverless: Runs inside your Python script with no external dependencies.

**Why Use DuckDB?**

-    It's faster than Pandas for complex queries.
-    Works well with large datasets without loading everything into memory.
-    No database setup required—just install and run!

We will demonstrate with the titanic example database.

**The Titanic dataset**

The Titanic Survival Dataset is one of the most famous datasets in data science and machine learning. It contains details about passengers aboard the RMS Titanic, which tragically sank on April 15, 1912, after hitting an iceberg. This dataset provides crucial insights into the survival patterns of passengers, helping analysts understand factors that influenced their chances of survival.

The dataset consists of passenger records, each containing demographic and travel-related information. Key attributes are the passenger class  (1 = First, 2 = Second, 3 = Third), survival (0 = no, 1 = yes), gender (male, female), age in years, fare paid for the ticket, port of embarkation (C = Cherbourg, Q = Queenstown, S = Southampton), and more.

Take a look at the example queries below. Study the SQL and the results. Later you will try to build the same functionality using pandas and matplotlib.

First, we will import duckdb, load the titanic dataset, count the records, and give an overview of the table.

In [2]:
import duckdb

print("Load full titanic dataset into duckdb as the titanic table")
duckdb.sql("CREATE TABLE titanic AS FROM 'https://gist.githubusercontent.com/teamtom/1af7b484954b2d4b7e981ea3e7a27f24/raw/114fb69dce56b4462a9c3a417e7402330616ad4f/titanic_full.csv';")

Load full titanic dataset into duckdb as the titanic table


CatalogException: Catalog Error: Table with name "titanic" already exists!

In [10]:
print("Count the number of records in the titanic table")
duckdb.sql("SELECT count(*) FROM titanic").show()

print("Give an overview of the titanic table")
duckdb.sql("SELECT * FROM titanic").show()

Count the number of records in the titanic table
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         1309 │
└──────────────┘

Give an overview of the titanic table
┌────────┬──────────┬──────────────────────────────────────────────────────────────┬─────────┬────────┬───────┬───────┬────────────┬──────────┬─────────────┬──────────┬─────────┬───────┬────────────────────────────┐
│ pclass │ survived │                             name                             │   sex   │  age   │ sibsp │ parch │   ticket   │   fare   │    cabin    │ embarked │  boat   │ body  │         home.dest          │
│ int64  │  int64   │                           varchar                            │ varchar │ double │ int64 │ int64 │  varchar   │  double  │   varchar   │ varchar  │ varchar │ int64 │          varchar           │
├────────┼──────────┼──────────────────────────────────────────────────────────────┼─────────┼────────┼───────┼───────┼────────────┼──────────┼─────────────┼─────

## Survival Rate by Sex

Were male or femal passengers more likely to survive?

In [3]:
print("Calculate the survival rate per gender")
duckdb.sql("""SELECT
    sex,
    AVG(survived) AS survival_rate
FROM
    titanic
GROUP BY
    sex;""").show()

Calculate the survival rate per gender
┌─────────┬─────────────────────┐
│   sex   │    survival_rate    │
│ varchar │       double        │
├─────────┼─────────────────────┤
│ female  │  0.7274678111587983 │
│ male    │ 0.19098457888493475 │
└─────────┴─────────────────────┘



## Survival per passnger class

Were first-class passengers more likely to survive?

In [4]:
print("Survival Rate by Passenger Class")
duckdb.sql("""
   SELECT
       pclass,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       pclass
   ORDER BY
       pclass;""").show()

Survival Rate by Passenger Class
┌────────┬────────────────────┐
│ pclass │   survival_rate    │
│ int64  │       double       │
├────────┼────────────────────┤
│      1 │ 0.6191950464396285 │
│      2 │ 0.4296028880866426 │
│      3 │ 0.2552891396332863 │
└────────┴────────────────────┘



## Survival Rate by Age Group

Did children have a higher survival rate than adults?

In [5]:
duckdb.sql("""
   SELECT
       CASE
           WHEN age < 10 THEN '0-9'
           WHEN age BETWEEN 10 AND 19 THEN '10-19'
           WHEN age BETWEEN 20 AND 29 THEN '20-29'
           WHEN age BETWEEN 30 AND 39 THEN '30-39'
           WHEN age BETWEEN 40 AND 49 THEN '40-49'
           WHEN age BETWEEN 50 AND 59 THEN '50-59'
           WHEN age >= 60 THEN '60+'
           ELSE 'Unknown'
       END AS age_group,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       age_group
   ORDER BY
       age_group;
""").show()

┌───────────┬─────────────────────┐
│ age_group │    survival_rate    │
│  varchar  │       double        │
├───────────┼─────────────────────┤
│ 0-9       │  0.6097560975609756 │
│ 10-19     │  0.3916083916083916 │
│ 20-29     │  0.3691860465116279 │
│ 30-39     │  0.4224137931034483 │
│ 40-49     │  0.3851851851851852 │
│ 50-59     │ 0.45714285714285713 │
│ 60+       │                 0.3 │
│ Unknown   │ 0.27756653992395436 │
└───────────┴─────────────────────┘



## Survival Rate by Embarkation Port

Did passengers from certain ports have a better chance of survival?

In [6]:
duckdb.sql("""
   SELECT
       embarked,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       embarked;
""").show()

┌──────────┬─────────────────────┐
│ embarked │    survival_rate    │
│ varchar  │       double        │
├──────────┼─────────────────────┤
│ NULL     │                 1.0 │
│ C        │  0.5555555555555556 │
│ Q        │ 0.35772357723577236 │
│ S        │ 0.33260393873085337 │
└──────────┴─────────────────────┘



### Survival Rate by Family Size

Did traveling with family improve survival chances?

In [7]:
duckdb.sql("""
   SELECT
       (sibsp + parch) AS family_size,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       family_size
   ORDER BY
       family_size;
""").show()

┌─────────────┬─────────────────────┐
│ family_size │    survival_rate    │
│    int64    │       double        │
├─────────────┼─────────────────────┤
│           0 │ 0.30253164556962026 │
│           1 │  0.5361702127659574 │
│           2 │  0.5660377358490566 │
│           3 │  0.6976744186046512 │
│           4 │  0.2727272727272727 │
│           5 │                 0.2 │
│           6 │                0.25 │
│           7 │                 0.0 │
│          10 │                 0.0 │
└─────────────┴─────────────────────┘



## Wealth and Survival (Fare Analysis)

Did passengers who paid more have a higher survival rate?

In [8]:
duckdb.sql("""
   SELECT
       CASE
           WHEN fare < 10 THEN 'Low Fare'
           WHEN fare BETWEEN 10 AND 50 THEN 'Medium Fare'
           WHEN fare > 50 THEN 'High Fare'
           ELSE 'Unknown'
       END AS fare_category,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       fare_category
   ORDER BY
       fare_category;
""").show()

┌───────────────┬────────────────────┐
│ fare_category │   survival_rate    │
│    varchar    │       double       │
├───────────────┼────────────────────┤
│ High Fare     │ 0.6583333333333333 │
│ Low Fare      │ 0.2240325865580448 │
│ Medium Fare   │ 0.4020797227036395 │
│ Unknown       │                0.0 │
└───────────────┴────────────────────┘



## Male vs. Female Survival Rate by Class

Was the "women and children first" rule followed across all classes?

In [9]:
duckdb.sql("""
   SELECT
       pclass,
       sex,
       AVG(survived) AS survival_rate
   FROM
       titanic
   GROUP BY
       pclass, sex
   ORDER BY
       pclass, sex;
""").show()

┌────────┬─────────┬─────────────────────┐
│ pclass │   sex   │    survival_rate    │
│ int64  │ varchar │       double        │
├────────┼─────────┼─────────────────────┤
│      1 │ female  │  0.9652777777777778 │
│      1 │ male    │  0.3407821229050279 │
│      2 │ female  │  0.8867924528301887 │
│      2 │ male    │ 0.14619883040935672 │
│      3 │ female  │ 0.49074074074074076 │
│      3 │ male    │ 0.15212981744421908 │
└────────┴─────────┴─────────────────────┘

